Importing utils

In [ ]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic

# load environment variable
load_dotenv()

# automatically looks for an "ANTHROPIC_API_KEY" environment variable
client = Anthropic()

Simple call

In [ ]:
conversation_assistant_log = []
conversation_history = []

while True:
    user_input = input("User: ")

    if user_input.lower() == "quit":
        print("Conversation ended.")
        break

    conversation_history.append({"role": "user", "content": user_input})

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        messages=conversation_history,
        system_prompt="You are a helpful assistant that can answer questions and help with tasks.",
        max_tokens=500,
        stop_sequences=[],
        temperature=0
    )

    assistant_response = response.content[0].text
    print(f"Assistant: {assistant_response}")

    # log the assistant response
    conversation_history.append({"role": "assistant", "content": assistant_response})
    conversation_assistant_log.append(f"Assistant: {assistant_response}")

Excercise 1

In [ ]:
import pprint


def generate_questions(topic="AI", number_of_questions=3):
    questions = []
    response = client.messages.create(
        model="claude-3-haiku-20240307",
        messages=[
            {
                "role": "user",
                "content": f"Based on the topic '{topic}', can you generate {number_of_questions}question for me? Only respond with the question, no other text. Generate a number list of questions",
            }
        ],
        system="You are a helpful assistant that can answer questions and help with tasks.",
        max_tokens=500,
        stop_sequences=["4"],
        stream=True
    )
    assistant_response = response.content[0].text
    questions.append(assistant_response)
    return questions

pprint.pprint(generate_questions())

Streaming 101

In [ ]:
def measuring_ttft_streming():
    stream = client.messages.create(
        messages=[
            {
                "role": "user",
                "content": "How do large language models work?",
            }
        ],
        model="claude-3-haiku-20240307",
        max_tokens=200,
        temperature=0,
        stream=True,
    )

    for event in stream:
        if event.type == "message_start":
            print("Input tokens :",event.message.usage.input_tokens, flush=True)
        elif event.type == "content_block_delta":
            print(event.delta.text, end="", flush=True)
        elif event.type == "message_delta":
            print("\n Output tokens:",event.usage.output_tokens, flush= True)

measuring_ttft_streming()

Streaming with helper functions

In [19]:
from anthropic import AsyncAnthropic

client = AsyncAnthropic()


async def streaming_with_helpers():

    conversation_history = []

    while True:
        user_input = input("User :")

        if user_input.lower == "quit":
            print("Conversation Ended")
            break

        conversation_history.append({"role": "user", "content": user_input})

        async with client.messages.stream(
            model="claude-3-5-haiku-20241022", max_tokens=200, messages=conversation_history
        ) as stream:
            async for text in stream.text_stream:
                print(text, end="", flush=True)

        assistant_final_response = await stream.get_final_message()
        assistant_text_response = assistant_final_response.content[0].text

        conversation_history.append(
            {"role": "assistant", "content": assistant_text_response}
        )

        print("\nconversation_history \n", conversation_history)


await streaming_with_helpers()

Hello there! How are you doing today?
conversation_history 
 [{'role': 'user', 'content': 'hi :)'}, {'role': 'assistant', 'content': 'Hello there! How are you doing today?'}]
I'm doing well, thanks for asking! I'm an AI assistant, so I aim to be helpful and friendly. Is there anything I can help you with today?
conversation_history 
 [{'role': 'user', 'content': 'hi :)'}, {'role': 'assistant', 'content': 'Hello there! How are you doing today?'}, {'role': 'user', 'content': 'Great ! How are you my friend ?'}, {'role': 'assistant', 'content': "I'm doing well, thanks for asking! I'm an AI assistant, so I aim to be helpful and friendly. Is there anything I can help you with today?"}]


BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.4: all messages must have non-empty content except for the optional final assistant message'}, 'request_id': 'req_011CVFYqqh7ArH6HLhBcEnqM'}

Structured Output

In [ ]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],
    messages=[
        {
            "role": "user",
            "content": "Extract the key information from this email: John Smith (john@example.com) is interested in our Enterprise plan and wants to schedule a demo for next Tuesday at 2pm.",
        }
    ],
    output_format={
        "type": "json_schema",
        "schema": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "plan_interest": {"type": "string"},
                "demo_requested": {"type": "boolean"},
            },
            "required": ["name", "email", "plan_interest", "demo_requested"],
            "additionalProperties": False,
        },
    },
)

In [ ]:
print(response.content[0].text)

In [ ]:
response

In [ ]:
print(f"stop_reason: {response.stop_reason}")
print(f"stop_sequence: {response.stop_sequence}")
print(f"input_tokens: {response.usage.input_tokens}")
print(f"output_tokens: {response.usage.output_tokens}")
print(f"server_tool_use: {response.usage.server_tool_use}")